In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
with open("datasets/september/harvest3r_twitter_data_01-09_0.json") as file:
    json = pd.read_json(file)

In [ ]:
json["_source"][1001]

In [ ]:
state_sentiment = []

for dictionary in json["_source"]:
    if "geo_state" in dictionary and "sentiment" in dictionary:
        state_sentiment.append([dictionary["geo_state"], dictionary["sentiment"]])

In [ ]:
len(state_sentiment) / len(json["_source"])

In [ ]:
state_sentiment_df = pd.DataFrame(state_sentiment, columns=["state", "sentiment"])

In [ ]:
state_sentiment_df.groupby("sentiment").count()

In [ ]:
state_sentiment_df["sentiment"][0:10]

In [ ]:
def sentiment_to_int(sentiment):
    if sentiment == "NEUTRAL":
        return 0
    elif sentiment == "POSITIVE":
        return 1
    elif sentiment == "NEGATIVE":
        return -1
    else:
        raise ValueError

state_sentiment_df["sentiment_int"] = state_sentiment_df["sentiment"].apply(sentiment_to_int)

In [ ]:
state_sentiment_df = state_sentiment_df.groupby("state").filter(lambda x: x.count()[0] > 10)

In [ ]:
state_sentiment_df.groupby("state").mean().sort_values(by="sentiment_int").plot(kind="bar")

In [ ]:
fields = ["geo_state", "main", "sentiment",  "author_gender", "geo_country", "geo_location", "published", ]
required_fields = ["geo_state", "sentiment"]

def json_to_df(path):
    with open(path) as file:
        json = pd.read_json(file)["_source"]
        
    filtered_json = []

    # Keep only relevant fields
    for j in json:
        filtered_json.append({field: j.get(field, None) for field in fields})
        
    # Filter out tweets witout state or sentiment
    dataframe = pd.DataFrame(filtered_json)
    
    dataframe.dropna(subset=required_fields, inplace=True)
    
    dataframe["sentiment_int"] = dataframe["sentiment"].apply(sentiment_to_int)
    
    return dataframe

In [ ]:
thursday_df = json_to_df("datasets/september/harvest3r_twitter_data_01-09_0.json")
friday_df = json_to_df("datasets/september/harvest3r_twitter_data_02-09_0.json")

In [ ]:
len(thursday_df)

In [ ]:
len(friday_df)

In [ ]:
thur = thursday_df.groupby("geo_state").mean()
fri = friday_df.groupby("geo_state").mean()

In [ ]:
df2 = pd.DataFrame()
df2["thursday"] = thur["sentiment_int"]
df2["friday"] = fri["sentiment_int"]

In [ ]:
# Are people happier on Friday than on Thursay? :)
df2

In [ ]:
thursday_df.groupby("author_gender").count()